In [1]:
import pandas as pd
import seaborn as sns

%load_ext google.cloud.bigquery

## Base Tables

In [2]:
params = {
    'report_start_date': '2023-01-01',
    'report_end_date': '2023-01-31'
}

In [3]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
-- TODO: Find a better way to write this with report_date included to show date of unsub
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Unsubs_v4_Jan23` AS (
    SELECT DISTINCT adobe_tracking_id
    FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE` 
    WHERE event_name = 'Email Unsubscribes' 
    AND event_date <= @report_end_date
)

Query is running:   0%|          |

""


In [4]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Targeted_v4_Jan23` AS ( -- everyone who have received emails in the month
    SELECT DISTINCT adobe_tracking_id 
    FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE` 
    WHERE event_name = 'Email Deliveries' 
    AND event_date BETWEEN @report_start_date and @report_end_date
    AND lower(campaign_name) NOT LIKE 'transactional%' -- Exclude transactional emails
)

Query is running:   0%|          |

""


In [5]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Holdout_v4_Jan23` AS ( 
    SELECT  distinct holdout.adobe_tracking_id
    FROM
    (
        SELECT  distinct TrackingId AS adobe_tracking_id -- distinct Hold_Out_Type_Current --cohort, count(distinct TrackingId)
        FROM `nbcu-ds-prod-001.PeacockDataMartMarketingGold.HOLDOUT_GROUP`
        WHERE cohort = format_timestamp('%B%Y', DATETIME_TRUNC(@report_start_date, QUARTER)) -- get cohort name as month of quarter start + year
        AND Hold_Out_Type_Current = 'Owned Email Holdout'
    ) holdout
    -- Exclude those who are assigned to Email Holdout but actually received emails 
    LEFT JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Targeted_v4_Jan23` targeted
    ON holdout.adobe_tracking_id = targeted.adobe_tracking_id
    WHERE targeted.adobe_tracking_id IS NULL
)

Query is running:   0%|          |

""


In [ ]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Engagers_and_New_Users_v4_Jan23` AS 

WITH 
email_engagers AS (
    -- Engagement: Deliveries 4 months before start of the holdout period, defined as start of quarter
    SELECT aid
    FROM
        (
            SELECT DISTINCT identity 
            FROM  `nbcu-sdp-prod-003.sdp_persistent_views.BrazeMarketingView` 
            WHERE eventName = 'Email Deliveries'
            AND date(eventTimestamp) BETWEEN DATE_SUB(DATETIME_TRUNC(@report_start_date, QUARTER), INTERVAL 4 MONTH) AND @report_end_date
            AND lower(campaignName) NOT LIKE 'transactional%' -- Exclude transactional emails
        ) engager_base
    INNER JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.Braze_Id_Adobe_Id_Map` mapping
    ON mapping.bid = engager_base.identity
)
SELECT aid
FROM email_engagers

UNION ALL

SELECT distinct adobe_tracking_id as aid
FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
WHERE registration_date >= DATE_SUB(DATETIME_TRUNC(@report_start_date, QUARTER), INTERVAL 4 MONTH)

### Output Base Tables

In [6]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.All_Cohorts_v4_Jan23` AS

SELECT  distinct a.adobe_tracking_id AS aid
       ,cohort
       ,user.account_type
       ,user.account_tenure
FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Engagers_and_New_Users_v4_Jan23` base

-- Include only those who received email in the current reporting period or are in holdout
INNER JOIN
    (
        SELECT *, 'Email_Targeted' as cohort from `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Targeted_v4_Jan23`
        UNION ALL
        SELECT *, 'Holdout' as cohort from `nbcu-ds-sandbox-a-001.SLi_sandbox.Holdout_v4_Jan23`
    ) a
ON a.adobe_tracking_id = base.aid

-- for after 2021/july, email channel only, take out all abandon MAAs
INNER JOIN
    (
        SELECT  DISTINCT adobe_tracking_id
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES`
        WHERE report_date = @report_end_date
        AND date_of_last_view IS NOT NULL
    ) abandon_maa
ON a.adobe_tracking_id = abandon_maa.adobe_tracking_id

-- exclude unsubscribed
LEFT JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Unsubs_v4_Jan23` unsub
ON a.adobe_tracking_id = unsub.adobe_tracking_id
WHERE unsub.adobe_tracking_id IS NULL

--add attribute: account_type at the end of the reporting period
INNER JOIN
    (
        SELECT  adobe_tracking_id
            ,account_type
            ,account_tenure
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
        WHERE report_date = @report_end_date 
    ) user
ON a.adobe_tracking_id = user.adobe_tracking_id

Executing query with job ID: b25b5fdb-98ef-4c5e-9611-d2e34e9661c4
Query executing: 0.24s


ERROR:
 400 Syntax error: Expected ")" but got end of script at [66:38]

Location: US
Job ID: b25b5fdb-98ef-4c5e-9611-d2e34e9661c4



In [7]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Video_Viewing_v4_Jan23` AS (
    SELECT 
        adobe_tracking_id
        , COUNT (DISTINCT CASE WHEN VIDEO.num_views_started = 1 THEN video.adobe_tracking_id ELSE NULL END) AS Distinct_Content_Starts
        , SUM (VIDEO.num_views_started ) AS Total_Content_Starts
        , SUM(VIDEO.num_seconds_played_no_ads)/3600 AS Viewing_Time
        , COUNT(DISTINCT session_id) AS Distinct_Viewing_Sessions 
        , COUNT(DISTINCT(CASE WHEN (num_seconds_played_no_ads > CASE WHEN lower(consumption_type) = 'virtual channel' THEN 299 ELSE 0 END)
                and (num_views_started>0) THEN CASE WHEN (lower(consumption_type) = "shortform") THEN "Shortform"
                                                    WHEN lower(franchise) != 'other' THEN franchise ELSE display_name
                                                    END
                end)) as Repertoire_Pavo_Method
                
    FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.All_Cohorts_v4_Jan23` a

    INNER JOIN  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_VIDEO` VIDEO
        ON VIDEO.adobe_tracking_id = a.aid
        AND adobe_date between @report_start_date AND @report_end_date
    GROUP BY 1
)

Query is running:   0%|          |

""


## Metric Definitions

In [8]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.CRM_KPI_Base_v4_Jan23` AS 

WITH
Save_Base AS ( --'Lapsing_Users'
    SELECT distinct adobe_tracking_id, date_of_last_view
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES` 
    WHERE report_date BETWEEN @report_start_date AND @report_end_date 
    AND days_since_last_view BETWEEN 15 AND 29 -- this guarantees we are only getting people who have at least past the 'lapsing' phase in the time period.
)
, Save_Denom as (
    SELECT distinct adobe_tracking_id
    FROM Save_Base
)
, Save_Num as (
    SELECT distinct a.adobe_tracking_id
    FROM Save_Base a
    INNER JOIN `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_VIDEO` VIDEO 
        ON a.adobe_tracking_id = VIDEO.adobe_tracking_id 
        AND adobe_date BETWEEN @report_start_date AND @report_end_date
        AND VIDEO.adobe_date BETWEEN DATE_ADD(date_of_last_view, INTERVAL 15 day) AND DATE_ADD(date_of_last_view, INTERVAL 29 day)
)
, Winback_Base AS ( --'Lapsed_Users'
    SELECT distinct adobe_tracking_id, date_of_last_view
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES` 
    WHERE report_date BETWEEN @report_start_date AND @report_end_date 
    AND days_since_last_view BETWEEN 30 AND 90 -- this guarantees we are only getting people who have at least past the 'lapsing' phase in the time period.
    )
, Winback_Denom AS (
    SELECT distinct adobe_tracking_id
    FROM Winback_Base
)
-- REVIEW: compare this code to email dash (Gold table)
, Winback_Num AS (
    SELECT distinct a.adobe_tracking_id
    FROM Winback_Base a
    INNER JOIN `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_VIDEO`  VIDEO 
        ON a.adobe_tracking_id = VIDEO.adobe_tracking_id 
        AND VIDEO.adobe_date BETWEEN @report_start_date AND @report_end_date 
        AND VIDEO.adobe_date BETWEEN DATE_ADD(date_of_last_view, INTERVAL 30 day) 
            AND DATE_ADD(date_of_last_view, INTERVAL 90 DAY)
)
, Upgrade_Denom AS (
-- upgrade metric 2.0: nonpaying to paying
    SELECT distinct adobe_tracking_id 
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`  USER
    WHERE paying_account_flag = 'NonPaying' 
    AND USER.report_date BETWEEN @report_start_date and @report_end_date
)
, Upgrade_Num AS (
-- upgrades from premium to premium+ would not be in the denom. What even is an upgrade - this definition differs from upgrade_rank table. TODO: Align with stakeholders to define upgrades
    SELECT  distinct adobe_tracking_id
    FROM
        (SELECT  
                report_date
             , adobe_tracking_id     
         FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`  USER
         WHERE entitlement_change_flag IN ('Upgrade: Free to Premium'
                                         , 'Upgrade: Free to Premium+')
                                         -- , 'Upgrade: Premium to Premium+'
         AND paying_account_flag = 'Paying'                                            
            AND USER.report_date BETWEEN @report_start_date and @report_end_date
        ) 
)    
, Cancel_Save_Denom AS (
    SELECT distinct adobe_tracking_id
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER` 
    WHERE paying_account_flag = 'Paying'
    AND auto_renew_flag = 'OFF'
    AND report_date BETWEEN @report_start_date and @report_end_date
)
, Cancel_Save_Num AS (
    SELECT  distinct adobe_tracking_id
    FROM
    (
        SELECT  adobe_tracking_id
            ,report_date
            ,auto_renew_flag                                                                      AS auto_renew_flag_today
            ,LEAD(auto_renew_flag,1) OVER ( partition by adobe_tracking_id ORDER BY report_date ) AS auto_renew_flag_next_day
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
        WHERE report_date BETWEEN @report_start_date AND @report_end_date
        ORDER BY 1, 2 
    )
    WHERE auto_renew_flag_today = 'OFF'
    AND auto_renew_flag_next_day = 'ON'
)
, New_Upgrade_Denom AS (
    SELECT  distinct adobe_tracking_id
    FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.upgrade_never`
    WHERE report_date BETWEEN @report_start_date AND @report_end_date
)
, New_Upgrade_Num AS (
    SELECT  distinct adobe_tracking_id
    FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.upgrade_date_rank`
    WHERE upgrade_row_number = 1
    AND report_date BETWEEN @report_start_date AND @report_end_date
)
, Paid_Winbacks_Denom AS (
    SELECT  distinct adobe_tracking_id
    FROM
    (
        SELECT  adobe_tracking_id
            ,report_date
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
        WHERE paying_account_flag = 'NonPaying'
        AND report_date BETWEEN @report_start_date AND @report_end_date 
    )
    WHERE adobe_tracking_id NOT IN ( SELECT distinct adobe_tracking_id FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.upgrade_never` WHERE report_date BETWEEN @report_start_date AND @report_end_date)
)
-- Super upgrades - paying base, from prem to prem+
, Paid_Winbacks_Num AS (
    SELECT  distinct adobe_tracking_id
    FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.upgrade_date_rank`
    WHERE upgrade_row_number > 1
    AND report_date BETWEEN @report_start_date AND @report_end_date
)
-- this metric uses silver_churn, which is based on data in PAVO dash Churn Trend
, EOM_Paid_Churn_Denom AS (
    SELECT  distinct adobe_tracking_id
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_CHURN`
    WHERE base_date = @report_end_date
    AND entitlement = 'Paid'
)
, EOM_Paid_Churn_Num AS (
    SELECT  distinct adobe_tracking_id
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_CHURN`
    WHERE base_date = @report_end_date
    AND entitlement = 'Paid'
    AND Churn_flag = 'Churn'
)
SELECT  @report_start_date                                                             AS Report_Month
       ,a.Account_Type
       ,a.Account_Tenure
       ,a.aid
       ,a.cohort
       ,CASE WHEN video.adobe_tracking_id IS NOT NULL THEN 1 ELSE 0 END                AS viewer
       ,video.Viewing_Time
       ,video.Repertoire_Pavo_Method
       ,video.Distinct_Viewing_Sessions
       --TODO: Split num and denom
       ,CASE 
            WHEN (Winback_Denom.adobe_tracking_id IS NOT NULL) AND (Winback_Num.adobe_tracking_id IS NOT NULL) THEN 1 
            WHEN (Winback_Denom.adobe_tracking_id IS NOT NULL) THEN 0
            ELSE NULL END 
        AS Winback
       ,CASE 
            WHEN (Save_Denom.adobe_tracking_id IS NOT NULL) AND (Save_Num.adobe_tracking_id IS NOT NULL) THEN 1 
            WHEN (Save_Denom.adobe_tracking_id IS NOT NULL) THEN 0
            ELSE NULL END 
        AS Save
       ,CASE 
            WHEN (Upgrade_Denom.adobe_tracking_id IS NOT NULL) AND (Upgrade_Num.adobe_tracking_id IS NOT NULL) THEN 1 
            WHEN (Upgrade_Denom.adobe_tracking_id IS NOT NULL) THEN 0
            ELSE NULL END 
        AS Upgrade
       ,CASE 
            WHEN (New_Upgrade_Denom.adobe_tracking_id IS NOT NULL) AND (New_Upgrade_Num.adobe_tracking_id IS NOT NULL) THEN 1 
            WHEN (New_Upgrade_Denom.adobe_tracking_id IS NOT NULL) THEN 0
            ELSE NULL END 
        AS New_Upgrade
       ,CASE 
            WHEN (Paid_Winbacks_Denom.adobe_tracking_id IS NOT NULL) AND (Paid_Winbacks_Num.adobe_tracking_id IS NOT NULL) THEN 1 
            WHEN (Paid_Winbacks_Denom.adobe_tracking_id IS NOT NULL) THEN 0
            ELSE NULL END 
        AS Paid_Winback
       ,CASE 
            WHEN (Cancel_Save_Denom.adobe_tracking_id IS NOT NULL) AND (Cancel_Save_Num.adobe_tracking_id IS NOT NULL) THEN 1 
            WHEN (Cancel_Save_Denom.adobe_tracking_id IS NOT NULL) THEN 0
            ELSE NULL END 
        AS Cancel_Save
       ,CASE 
            WHEN (EOM_Paid_Churn_Denom.adobe_tracking_id IS NOT NULL) AND (EOM_Paid_Churn_Num.adobe_tracking_id IS NOT NULL) THEN 1 
            WHEN (EOM_Paid_Churn_Denom.adobe_tracking_id IS NOT NULL) THEN 0
            ELSE NULL END 
        AS EOM_Paid_Churn
FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.All_Cohorts_v4_Jan23` a
LEFT JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.Video_Viewing_v4_Jan23` video
    ON a.aid = video.adobe_tracking_id
LEFT JOIN Winback_Denom
    ON a.aid = Winback_Denom.adobe_tracking_id
LEFT JOIN Winback_Num
    ON Winback_Denom.adobe_tracking_id = Winback_Num.adobe_tracking_id
LEFT JOIN Save_Denom
    ON a.aid = Save_Denom.adobe_tracking_id
LEFT JOIN Save_Num
    ON Save_Denom.adobe_tracking_id = Save_Num.adobe_tracking_id
LEFT JOIN Upgrade_Denom
    ON a.aid = Upgrade_Denom.adobe_tracking_id
LEFT JOIN Upgrade_Num 
    ON Upgrade_Denom.adobe_tracking_id = Upgrade_Num.adobe_tracking_id
LEFT JOIN New_Upgrade_Denom
    on a.aid = New_Upgrade_Denom.adobe_tracking_id
LEFT JOIN New_Upgrade_Num
    on New_Upgrade_Denom.adobe_tracking_id = New_Upgrade_Num.adobe_tracking_id
LEFT JOIN Paid_Winbacks_Denom
    on a.aid = Paid_Winbacks_Denom.adobe_tracking_id
LEFT JOIN Paid_Winbacks_Num
    on Paid_Winbacks_Num.adobe_tracking_id = Paid_Winbacks_Denom.adobe_tracking_id
LEFT JOIN Cancel_Save_Denom
    on a.aid = Cancel_Save_Denom.adobe_tracking_id
LEFT JOIN Cancel_Save_Num
    on Cancel_Save_Denom.adobe_tracking_id = Cancel_Save_Num.adobe_tracking_id
LEFT JOIN EOM_Paid_Churn_Denom
    on a.aid = EOM_Paid_Churn_Denom.adobe_tracking_id
LEFT JOIN EOM_Paid_Churn_Num
    on EOM_Paid_Churn_Denom.adobe_tracking_id = EOM_Paid_Churn_Num.adobe_tracking_id

Executing query with job ID: 1c447cab-5f21-4b33-bf72-a4bb71aaf2ca
Query executing: 5566.29s


ERROR:
 HTTPSConnectionPool(host='oauth2.googleapis.com', port=443): Max retries exceeded with url: /token (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001FC4EC428B0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))


# Analysis

In [14]:
%%bigquery df_total --project nbcu-ds-sandbox-a-001 --params $params

WITH 
CTE_1 AS (
    SELECT  @report_start_date                                                           AS Report_Month
       --, Account_Type
           ,COUNT(distinct CASE WHEN cohort = 'Email_Targeted' THEN aid END )            AS Distinct_Cohort_Size_Targeted
           ,COUNT(distinct CASE WHEN cohort = 'Holdout' THEN aid END)                    AS Distinct_Cohort_Size_Holdout
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN viewer END )                   AS Total_Returns_Targeted
           ,SUM(CASE WHEN cohort = 'Holdout' THEN viewer END)                           AS Total_Returns_Holdout
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Viewing_Time END )              AS Total_Usage_Targeted
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Viewing_Time END)                      AS Total_Usage_Holdout
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Repertoire_Pavo_Method END )    AS Total_Repertoire_Targeted
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Repertoire_Pavo_Method END)            AS Total_Repertoire_Holdout
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Distinct_Viewing_Sessions END ) AS Total_Viewing_Sessions_Targeted
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Distinct_Viewing_Sessions END)         AS Total_Viewing_Sessions_Holdout
           ,COUNT(CASE WHEN cohort = 'Email_Targeted' THEN Winback END )             AS Winback_Denom_Targeted
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Winback END)                AS Winback_Num_Targeted
           ,COUNT(CASE WHEN cohort = 'Holdout' THEN Winback END )                    AS Winback_Denom_Holdout
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Winback END)                       AS Winback_Num_Holdout
           ,COUNT(CASE WHEN cohort = 'Email_Targeted' THEN Save END )                AS Save_Denom_Targeted
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Save END)                   AS Save_Num_Targeted
           ,COUNT(CASE WHEN cohort = 'Holdout' THEN Save END )                       AS Save_Denom_Holdout
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Save END)                          AS Save_Num_Holdout
           ,COUNT(CASE WHEN cohort = 'Email_Targeted' THEN Upgrade END )             AS Upgrades_Denom_Targeted
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Upgrade END)                AS Upgrades_Num_Targeted
           ,COUNT(CASE WHEN cohort = 'Holdout' THEN Upgrade END )                    AS Upgrades_Denom_Holdout
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Upgrade END)                       AS Upgrades_Num_Holdout
           ,COUNT(CASE WHEN cohort = 'Email_Targeted' THEN New_Upgrade END )         AS Total_New_Upgrade_Denom_Targeted
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN New_Upgrade END)            AS Total_New_Upgrade_Num_Targeted
           ,COUNT(CASE WHEN cohort = 'Holdout' THEN New_Upgrade END )                AS Total_New_Upgrade_Denom_Holdout
           ,SUM(CASE WHEN cohort = 'Holdout' THEN New_Upgrade END)                   AS Total_New_Upgrade_Num_Holdout
           ,COUNT(CASE WHEN cohort = 'Email_Targeted' THEN Paid_Winback END )       AS Total_Paid_Winbacks_Denom_Targeted
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Paid_Winback END)          AS Total_Paid_Winbacks_Num_Targeted
           ,COUNT(CASE WHEN cohort = 'Holdout' THEN Paid_Winback END )              AS Total_Paid_Winbacks_Denom_Holdout
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Paid_Winback END)                 AS Total_Paid_Winbacks_Num_Holdout
           ,COUNT(CASE WHEN cohort = 'Email_Targeted' THEN Cancel_Save END )         AS Total_Cancel_Save_Denom_Targeted
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Cancel_Save END)            AS Total_Cancel_Save_Num_Targeted
           ,COUNT(CASE WHEN cohort = 'Holdout' THEN Cancel_Save END )                AS Total_Cancel_Save_Denom_Holdout
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Cancel_Save END)                   AS Total_Cancel_Save_Num_Holdout
           ,COUNT(CASE WHEN cohort = 'Email_Targeted' THEN EOM_Paid_Churn END )      AS EOM_Paid_Churn_Denom_Targeted
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN EOM_Paid_Churn END)         AS EOM_Paid_Churn_Num_Targeted
           ,COUNT(CASE WHEN cohort = 'Holdout' THEN EOM_Paid_Churn END )             AS EOM_Paid_Churn_Denom_Holdout
           ,SUM(CASE WHEN cohort = 'Holdout' THEN EOM_Paid_Churn END)                AS EOM_Paid_Churn_Num_Holdout
       FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.CRM_KPI_Base_v4_Jan23`
       GROUP BY  1 --, 2

), CTE_2 AS (
    SELECT  Report_Month
            -- ,Account_Type
            ,Distinct_Cohort_Size_Targeted                                                            AS Email_Engagers
            ,Distinct_Cohort_Size_Holdout                                                             AS Email_Holdout
            ,safe_divide(Total_Returns_Targeted,Distinct_Cohort_Size_Targeted)                        AS Return_Rate_Engagers
            ,safe_divide(Total_Returns_Holdout,Distinct_Cohort_Size_Holdout)                          AS Return_Rate_Holdout
            ,safe_divide(Total_Usage_Targeted,Distinct_Cohort_Size_Targeted)                          AS Usage_Engagers
            ,safe_divide(Total_Usage_Holdout,Distinct_Cohort_Size_Holdout)                            AS Usage_Holdout
            ,safe_divide(Total_Repertoire_Targeted,Distinct_Cohort_Size_Targeted)                     AS Repertoire_Engagers
            ,safe_divide(Total_Repertoire_Holdout,Distinct_Cohort_Size_Holdout)                       AS Repertoire_Holdout
            ,safe_divide(Total_Viewing_Sessions_Targeted,Distinct_Cohort_Size_Targeted)               AS Sessions_Engagers
            ,safe_divide(Total_Viewing_Sessions_Holdout,Distinct_Cohort_Size_Holdout)                 AS Sessions_Holdout
            ,Winback_Denom_Targeted
            ,safe_divide(Winback_Num_Targeted,Winback_Denom_Targeted)                                 AS Winback_Rate_Engagers
            ,safe_divide(Winback_Num_Holdout,Winback_Denom_Holdout)                                   AS Winback_Rate_Holdout
            ,Save_Denom_Targeted
            ,safe_divide(Save_Num_Targeted,Save_Denom_Targeted)                                       AS Save_Rate_Engagers
            ,safe_divide(Save_Num_Holdout,Save_Denom_Holdout)                                         AS Save_Rate_Holdout
            ,Upgrades_Denom_Targeted
            ,safe_divide(Upgrades_Num_Targeted,Upgrades_Denom_Targeted)                               AS Upgrade_Rate_Engagers
            ,safe_divide(Upgrades_Num_Holdout,Upgrades_Denom_Holdout)                                 AS Upgrade_Rate_Holdout
            ,Total_New_Upgrade_Denom_Targeted
            ,safe_divide(Total_New_Upgrade_Num_Targeted,Total_New_Upgrade_Denom_Targeted)             AS New_Upgrade_Rate_Engagers
            ,safe_divide(Total_New_Upgrade_Num_Holdout,Total_New_Upgrade_Denom_Holdout)               AS New_Upgrade_Rate_Holdout
            ,Total_Paid_Winbacks_Denom_Targeted
            ,safe_divide(Total_Paid_Winbacks_Num_Targeted,Total_Paid_Winbacks_Denom_Targeted)         AS Paid_Winback_Rate_Engagers
            ,safe_divide(Total_Paid_Winbacks_Num_Holdout,Total_Paid_Winbacks_Denom_Holdout)           AS Paid_Winback_Rate_Holdout
            ,Total_Cancel_Save_Denom_Targeted
            ,safe_divide(Total_Cancel_Save_Num_Targeted,Total_Cancel_Save_Denom_Targeted)             AS Cancel_Save_Rate_Engagers
            ,safe_divide(Total_Cancel_Save_Num_Holdout,Total_Cancel_Save_Denom_Holdout)               AS Cancel_Save_Rate_Holdout
            ,EOM_Paid_Churn_Denom_Targeted
            ,safe_divide(EOM_Paid_Churn_Num_Targeted,EOM_Paid_Churn_Denom_Targeted)                   AS Paid_Churn_Rate_Engagers
            ,safe_divide(EOM_Paid_Churn_Num_Holdout,EOM_Paid_Churn_Denom_Holdout)                     AS Paid_Churn_Rate_Holdout
    FROM CTE_1
)

SELECT  Report_Month
       --,Account_Type
       ,Email_Engagers
       ,Email_Holdout

       ,Return_Rate_Engagers                                                                                        AS Return_Rate_Engagers
       ,Return_Rate_Holdout                                                                                         AS Return_Rate_Holdout
       ,Return_Rate_Engagers - Return_Rate_Holdout                                                                  AS Return_Rate_Lift_PTS
       ,safe_divide(Return_Rate_Engagers,Return_Rate_Holdout) *100                                                  AS Return_Rate_Lift_Index
       ,(Return_Rate_Engagers - Return_Rate_Holdout) * Email_Engagers                                               AS Returns_Incrementals

       ,Usage_Engagers                                                                                              AS Usage_Engagers
       ,Usage_Holdout                                                                                               AS Usage_Holdout
       ,Usage_Engagers - Usage_Holdout                                                                              AS Usage_Lift_PTS
       ,safe_divide(Usage_Engagers,Usage_Holdout) *100                                                              AS Usage_Lift_Index
       ,(Usage_Engagers - Usage_Holdout) * Email_Engagers                                                           AS Usage_Incrementals

       ,Repertoire_Engagers                                                                                         AS Repertoire_Engagers
       ,Repertoire_Holdout                                                                                          AS Repertoire_Holdout
       ,Repertoire_Engagers - Repertoire_Holdout                                                                    AS Repertoire_Lift_PTS
       ,safe_divide(Repertoire_Engagers,Repertoire_Holdout) *100                                                    AS Repertoire_Lift_Index
       ,(Repertoire_Engagers - Repertoire_Holdout) * Email_Engagers                                                 AS Repertoire_Incrementals

       ,Sessions_Engagers                                                                                           AS Sessions_Engagers
       ,Sessions_Holdout                                                                                            AS Sessions_Holdout
       ,Sessions_Engagers - Sessions_Holdout                                                                        AS Sessions_Lift_PTS
       ,safe_divide(Sessions_Engagers,Sessions_Holdout) *100                                                        AS Sessions_Lift_Index
       ,(Sessions_Engagers - Sessions_Holdout) * Email_Engagers                                                     AS Sessions_Incrementals

       ,Winback_Rate_Engagers                                                                                       AS Winback_Rate_Engagers
       ,Winback_Rate_Holdout                                                                                        AS Winback_Rate_Holdout
       ,Winback_Rate_Engagers - Winback_Rate_Holdout                                                                AS Winback_Rate_Lift_PTS
       ,safe_divide(Winback_Rate_Engagers,Winback_Rate_Holdout) *100                                                AS Winback_Rate_Lift_Index
       ,(Winback_Rate_Engagers - Winback_Rate_Holdout) * Winback_Denom_Targeted                                     AS Winback_Incrementals

       ,Save_Rate_Engagers                                                                                          AS Save_Rate_Engagers
       ,Save_Rate_Holdout                                                                                           AS Save_Rate_Holdout
       ,Save_Rate_Engagers - Save_Rate_Holdout                                                                      AS Save_Rate_Lift_PTS
       ,safe_divide(Save_Rate_Engagers,Save_Rate_Holdout) *100                                                      AS Save_Rate_Lift_Index
       ,(Save_Rate_Engagers - Save_Rate_Holdout) * Save_Denom_Targeted                                              AS Save_Rate_Lift_Incrementals

       ,Upgrade_Rate_Engagers                                                                                       AS Upgrade_Rate_Engagers
       ,Upgrade_Rate_Holdout                                                                                        AS Upgrade_Rate_Holdout
       ,Upgrade_Rate_Engagers - Upgrade_Rate_Holdout                                                                AS Upgrade_Rate_Lift_PTS
       ,safe_divide(Upgrade_Rate_Engagers,Upgrade_Rate_Holdout) *100                                                AS Upgrade_Rate_Lift_Index
       ,(Upgrade_Rate_Engagers - Upgrade_Rate_Holdout) * Upgrades_Denom_Targeted                                    AS Upgrade_Incrementals

       ,New_Upgrade_Rate_Engagers                                                                                   AS New_Upgrade_Rate_Engagers
       ,New_Upgrade_Rate_Holdout                                                                                    AS New_Upgrade_Rate_Holdout
       ,New_Upgrade_Rate_Engagers - New_Upgrade_Rate_Holdout                                                        AS New_Upgrade_Rate_Lift_PTS
       ,safe_divide(New_Upgrade_Rate_Engagers,New_Upgrade_Rate_Holdout) *100                                        AS New_Upgrade_Rate_Lift_Index
       ,(New_Upgrade_Rate_Engagers - New_Upgrade_Rate_Holdout) * Total_New_Upgrade_Denom_Targeted                   AS New_Upgrade_Incrementals

       ,Paid_Winback_Rate_Engagers                                                                                  AS Paid_Winback_Rate_Engagers
       ,Paid_Winback_Rate_Holdout                                                                                   AS Paid_Winback_Rate_Holdout
       ,Paid_Winback_Rate_Engagers - Paid_Winback_Rate_Holdout                                                      AS Paid_Winback_Rate_Lift_PTS
       ,safe_divide(Paid_Winback_Rate_Engagers,Paid_Winback_Rate_Holdout) *100                                      AS Paid_Winback_Rate_Lift_Index
       ,(Paid_Winback_Rate_Engagers - Paid_Winback_Rate_Holdout) * Total_Paid_Winbacks_Denom_Targeted               AS Paid_Winback_Rate_Lift_Incrementals

       ,Cancel_Save_Rate_Engagers                                                                             AS Cancel_Save_Rate_Engagers
       ,Cancel_Save_Rate_Holdout                                                                              AS Cancel_Save_Rate_Holdout
       ,Cancel_Save_Rate_Engagers - Cancel_Save_Rate_Holdout                                            AS Cancel_Save_Rate_Lift_PTS
       ,safe_divide(Cancel_Save_Rate_Engagers,Cancel_Save_Rate_Holdout) *100                            AS Cancel_Save_Rate_Lift_Index
       ,(Cancel_Save_Rate_Engagers - Cancel_Save_Rate_Holdout) * Total_Cancel_Save_Denom_Targeted     AS Cancel_Save_Rate_Incrementals

       ,Paid_Churn_Rate_Engagers                                                                                    AS EOM_Paid_Churn_Rate_Engagers
       ,Paid_Churn_Rate_Holdout                                                                                     AS EOM_Paid_Churn_Rate_Holdout
       ,Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout                                                          AS EOM_Paid_Churn_Rate_Lift_PTS
       ,safe_divide(Paid_Churn_Rate_Engagers,Paid_Churn_Rate_Holdout) *100                                          AS EOM_Paid_Churn_Rate_Lift_Index
       ,(Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout) * EOM_Paid_Churn_Denom_Targeted                        AS EOM_Paid_Churn_Rate_Incrementals

FROM CTE_2

Query is running:   0%|          |

Downloading:   0%|          |

In [15]:
df_total

,Report_Month,Email_Engagers,Email_Holdout,Return_Rate_Engagers,Return_Rate_Holdout,Return_Rate_Lift_PTS,Return_Rate_Lift_Index,Returns_Incrementals,Usage_Engagers,Usage_Holdout,...,Cancel_Save_Rate_Engagers,Cancel_Save_Rate_Holdout,Cancel_Save_Rate_Lift_PTS,Cancel_Save_Rate_Lift_Index,Cancel_Save_Rate_Incrementals,EOM_Paid_Churn_Rate_Engagers,EOM_Paid_Churn_Rate_Holdout,EOM_Paid_Churn_Rate_Lift_PTS,EOM_Paid_Churn_Rate_Lift_Index,EOM_Paid_Churn_Rate_Incrementals
0,2023-01-01,15951985,653595,0.459369,0.402375,0.056994,114.164502,909173.067756,8.776216,7.542804,...,0.02815,0.023304,0.004846,120.79513,1071.130088,0.065712,0.058593,0.007119,112.150113,34654.385046


In [16]:
%%bigquery df_acc --project nbcu-ds-sandbox-a-001 --params $params

WITH 
CTE_1 AS (
    SELECT  @report_start_date                                                           AS Report_Month
           ,Account_Type
           ,COUNT(distinct CASE WHEN cohort = 'Email_Targeted' THEN aid END )            AS Distinct_Cohort_Size_Targeted
           ,COUNT(distinct CASE WHEN cohort = 'Holdout' THEN aid END)                    AS Distinct_Cohort_Size_Holdout
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN viewer END )                   AS Total_Returns_Targeted
           ,SUM(CASE WHEN cohort = 'Holdout' THEN viewer END)                           AS Total_Returns_Holdout
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Viewing_Time END )              AS Total_Usage_Targeted
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Viewing_Time END)                      AS Total_Usage_Holdout
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Repertoire_Pavo_Method END )    AS Total_Repertoire_Targeted
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Repertoire_Pavo_Method END)            AS Total_Repertoire_Holdout
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Distinct_Viewing_Sessions END ) AS Total_Viewing_Sessions_Targeted
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Distinct_Viewing_Sessions END)         AS Total_Viewing_Sessions_Holdout
           ,COUNT(CASE WHEN cohort = 'Email_Targeted' THEN Winback END )             AS Winback_Denom_Targeted
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Winback END)                AS Winback_Num_Targeted
           ,COUNT(CASE WHEN cohort = 'Holdout' THEN Winback END )                    AS Winback_Denom_Holdout
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Winback END)                       AS Winback_Num_Holdout
           ,COUNT(CASE WHEN cohort = 'Email_Targeted' THEN Save END )                AS Save_Denom_Targeted
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Save END)                   AS Save_Num_Targeted
           ,COUNT(CASE WHEN cohort = 'Holdout' THEN Save END )                       AS Save_Denom_Holdout
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Save END)                          AS Save_Num_Holdout
           ,COUNT(CASE WHEN cohort = 'Email_Targeted' THEN Upgrade END )             AS Upgrades_Denom_Targeted
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Upgrade END)                AS Upgrades_Num_Targeted
           ,COUNT(CASE WHEN cohort = 'Holdout' THEN Upgrade END )                    AS Upgrades_Denom_Holdout
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Upgrade END)                       AS Upgrades_Num_Holdout
           ,COUNT(CASE WHEN cohort = 'Email_Targeted' THEN New_Upgrade END )         AS Total_New_Upgrade_Denom_Targeted
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN New_Upgrade END)            AS Total_New_Upgrade_Num_Targeted
           ,COUNT(CASE WHEN cohort = 'Holdout' THEN New_Upgrade END )                AS Total_New_Upgrade_Denom_Holdout
           ,SUM(CASE WHEN cohort = 'Holdout' THEN New_Upgrade END)                   AS Total_New_Upgrade_Num_Holdout
           ,COUNT(CASE WHEN cohort = 'Email_Targeted' THEN Paid_Winback END )       AS Total_Paid_Winbacks_Denom_Targeted
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Paid_Winback END)          AS Total_Paid_Winbacks_Num_Targeted
           ,COUNT(CASE WHEN cohort = 'Holdout' THEN Paid_Winback END )              AS Total_Paid_Winbacks_Denom_Holdout
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Paid_Winback END)                 AS Total_Paid_Winbacks_Num_Holdout
           ,COUNT(CASE WHEN cohort = 'Email_Targeted' THEN Cancel_Save END )         AS Total_Cancel_Save_Denom_Targeted
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Cancel_Save END)            AS Total_Cancel_Save_Num_Targeted
           ,COUNT(CASE WHEN cohort = 'Holdout' THEN Cancel_Save END )                AS Total_Cancel_Save_Denom_Holdout
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Cancel_Save END)                   AS Total_Cancel_Save_Num_Holdout
           ,COUNT(CASE WHEN cohort = 'Email_Targeted' THEN EOM_Paid_Churn END )      AS EOM_Paid_Churn_Denom_Targeted
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN EOM_Paid_Churn END)         AS EOM_Paid_Churn_Num_Targeted
           ,COUNT(CASE WHEN cohort = 'Holdout' THEN EOM_Paid_Churn END )             AS EOM_Paid_Churn_Denom_Holdout
           ,SUM(CASE WHEN cohort = 'Holdout' THEN EOM_Paid_Churn END)                AS EOM_Paid_Churn_Num_Holdout
       FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.CRM_KPI_Base_v4_Jan23`
       GROUP BY  1, 2

), CTE_2 AS (
    SELECT  Report_Month
            ,Account_Type
            ,Distinct_Cohort_Size_Targeted                                                            AS Email_Engagers
            ,Distinct_Cohort_Size_Holdout                                                             AS Email_Holdout
            ,safe_divide(Total_Returns_Targeted,Distinct_Cohort_Size_Targeted)                        AS Return_Rate_Engagers
            ,safe_divide(Total_Returns_Holdout,Distinct_Cohort_Size_Holdout)                          AS Return_Rate_Holdout
            ,safe_divide(Total_Usage_Targeted,Distinct_Cohort_Size_Targeted)                          AS Usage_Engagers
            ,safe_divide(Total_Usage_Holdout,Distinct_Cohort_Size_Holdout)                            AS Usage_Holdout
            ,safe_divide(Total_Repertoire_Targeted,Distinct_Cohort_Size_Targeted)                     AS Repertoire_Engagers
            ,safe_divide(Total_Repertoire_Holdout,Distinct_Cohort_Size_Holdout)                       AS Repertoire_Holdout
            ,safe_divide(Total_Viewing_Sessions_Targeted,Distinct_Cohort_Size_Targeted)               AS Sessions_Engagers
            ,safe_divide(Total_Viewing_Sessions_Holdout,Distinct_Cohort_Size_Holdout)                 AS Sessions_Holdout
            ,Winback_Denom_Targeted
            ,safe_divide(Winback_Num_Targeted,Winback_Denom_Targeted)                                 AS Winback_Rate_Engagers
            ,safe_divide(Winback_Num_Holdout,Winback_Denom_Holdout)                                   AS Winback_Rate_Holdout
            ,Save_Denom_Targeted
            ,safe_divide(Save_Num_Targeted,Save_Denom_Targeted)                                       AS Save_Rate_Engagers
            ,safe_divide(Save_Num_Holdout,Save_Denom_Holdout)                                         AS Save_Rate_Holdout
            ,Upgrades_Denom_Targeted
            ,safe_divide(Upgrades_Num_Targeted,Upgrades_Denom_Targeted)                               AS Upgrade_Rate_Engagers
            ,safe_divide(Upgrades_Num_Holdout,Upgrades_Denom_Holdout)                                 AS Upgrade_Rate_Holdout
            ,Total_New_Upgrade_Denom_Targeted
            ,safe_divide(Total_New_Upgrade_Num_Targeted,Total_New_Upgrade_Denom_Targeted)             AS New_Upgrade_Rate_Engagers
            ,safe_divide(Total_New_Upgrade_Num_Holdout,Total_New_Upgrade_Denom_Holdout)               AS New_Upgrade_Rate_Holdout
            ,Total_Paid_Winbacks_Denom_Targeted
            ,safe_divide(Total_Paid_Winbacks_Num_Targeted,Total_Paid_Winbacks_Denom_Targeted)         AS Paid_Winback_Rate_Engagers
            ,safe_divide(Total_Paid_Winbacks_Num_Holdout,Total_Paid_Winbacks_Denom_Holdout)           AS Paid_Winback_Rate_Holdout
            ,Total_Cancel_Save_Denom_Targeted
            ,safe_divide(Total_Cancel_Save_Num_Targeted,Total_Cancel_Save_Denom_Targeted)             AS Cancel_Save_Rate_Engagers
            ,safe_divide(Total_Cancel_Save_Num_Holdout,Total_Cancel_Save_Denom_Holdout)               AS Cancel_Save_Rate_Holdout
            ,EOM_Paid_Churn_Denom_Targeted
            ,safe_divide(EOM_Paid_Churn_Num_Targeted,EOM_Paid_Churn_Denom_Targeted)                   AS Paid_Churn_Rate_Engagers
            ,safe_divide(EOM_Paid_Churn_Num_Holdout,EOM_Paid_Churn_Denom_Holdout)                     AS Paid_Churn_Rate_Holdout
    FROM CTE_1
)

SELECT  Report_Month
       ,Account_Type
       ,Email_Engagers
       ,Email_Holdout

       ,Return_Rate_Engagers                                                                                        AS Return_Rate_Engagers
       ,Return_Rate_Holdout                                                                                         AS Return_Rate_Holdout
       ,Return_Rate_Engagers - Return_Rate_Holdout                                                                  AS Return_Rate_Lift_PTS
       ,safe_divide(Return_Rate_Engagers,Return_Rate_Holdout) *100                                                  AS Return_Rate_Lift_Index
       ,(Return_Rate_Engagers - Return_Rate_Holdout) * Email_Engagers                                               AS Returns_Incrementals

       ,Usage_Engagers                                                                                              AS Usage_Engagers
       ,Usage_Holdout                                                                                               AS Usage_Holdout
       ,Usage_Engagers - Usage_Holdout                                                                              AS Usage_Lift_PTS
       ,safe_divide(Usage_Engagers,Usage_Holdout) *100                                                              AS Usage_Lift_Index
       ,(Usage_Engagers - Usage_Holdout) * Email_Engagers                                                           AS Usage_Incrementals

       ,Repertoire_Engagers                                                                                         AS Repertoire_Engagers
       ,Repertoire_Holdout                                                                                          AS Repertoire_Holdout
       ,Repertoire_Engagers - Repertoire_Holdout                                                                    AS Repertoire_Lift_PTS
       ,safe_divide(Repertoire_Engagers,Repertoire_Holdout) *100                                                    AS Repertoire_Lift_Index
       ,(Repertoire_Engagers - Repertoire_Holdout) * Email_Engagers                                                 AS Repertoire_Incrementals

       ,Sessions_Engagers                                                                                           AS Sessions_Engagers
       ,Sessions_Holdout                                                                                            AS Sessions_Holdout
       ,Sessions_Engagers - Sessions_Holdout                                                                        AS Sessions_Lift_PTS
       ,safe_divide(Sessions_Engagers,Sessions_Holdout) *100                                                        AS Sessions_Lift_Index
       ,(Sessions_Engagers - Sessions_Holdout) * Email_Engagers                                                     AS Sessions_Incrementals

       ,Winback_Rate_Engagers                                                                                       AS Winback_Rate_Engagers
       ,Winback_Rate_Holdout                                                                                        AS Winback_Rate_Holdout
       ,Winback_Rate_Engagers - Winback_Rate_Holdout                                                                AS Winback_Rate_Lift_PTS
       ,safe_divide(Winback_Rate_Engagers,Winback_Rate_Holdout) *100                                                AS Winback_Rate_Lift_Index
       ,(Winback_Rate_Engagers - Winback_Rate_Holdout) * Winback_Denom_Targeted                                     AS Winback_Incrementals

       ,Save_Rate_Engagers                                                                                          AS Save_Rate_Engagers
       ,Save_Rate_Holdout                                                                                           AS Save_Rate_Holdout
       ,Save_Rate_Engagers - Save_Rate_Holdout                                                                      AS Save_Rate_Lift_PTS
       ,safe_divide(Save_Rate_Engagers,Save_Rate_Holdout) *100                                                      AS Save_Rate_Lift_Index
       ,(Save_Rate_Engagers - Save_Rate_Holdout) * Save_Denom_Targeted                                              AS Save_Rate_Lift_Incrementals

       ,Upgrade_Rate_Engagers                                                                                       AS Upgrade_Rate_Engagers
       ,Upgrade_Rate_Holdout                                                                                        AS Upgrade_Rate_Holdout
       ,Upgrade_Rate_Engagers - Upgrade_Rate_Holdout                                                                AS Upgrade_Rate_Lift_PTS
       ,safe_divide(Upgrade_Rate_Engagers,Upgrade_Rate_Holdout) *100                                                AS Upgrade_Rate_Lift_Index
       ,(Upgrade_Rate_Engagers - Upgrade_Rate_Holdout) * Upgrades_Denom_Targeted                                    AS Upgrade_Incrementals

       ,New_Upgrade_Rate_Engagers                                                                                   AS New_Upgrade_Rate_Engagers
       ,New_Upgrade_Rate_Holdout                                                                                    AS New_Upgrade_Rate_Holdout
       ,New_Upgrade_Rate_Engagers - New_Upgrade_Rate_Holdout                                                        AS New_Upgrade_Rate_Lift_PTS
       ,safe_divide(New_Upgrade_Rate_Engagers,New_Upgrade_Rate_Holdout) *100                                        AS New_Upgrade_Rate_Lift_Index
       ,(New_Upgrade_Rate_Engagers - New_Upgrade_Rate_Holdout) * Total_New_Upgrade_Denom_Targeted                   AS New_Upgrade_Incrementals

       ,Paid_Winback_Rate_Engagers                                                                                  AS Paid_Winback_Rate_Engagers
       ,Paid_Winback_Rate_Holdout                                                                                   AS Paid_Winback_Rate_Holdout
       ,Paid_Winback_Rate_Engagers - Paid_Winback_Rate_Holdout                                                      AS Paid_Winback_Rate_Lift_PTS
       ,safe_divide(Paid_Winback_Rate_Engagers,Paid_Winback_Rate_Holdout) *100                                      AS Paid_Winback_Rate_Lift_Index
       ,(Paid_Winback_Rate_Engagers - Paid_Winback_Rate_Holdout) * Total_Paid_Winbacks_Denom_Targeted               AS Paid_Winback_Rate_Lift_Incrementals

       ,Cancel_Save_Rate_Engagers                                                                             AS Cancel_Save_Rate_Engagers
       ,Cancel_Save_Rate_Holdout                                                                              AS Cancel_Save_Rate_Holdout
       ,Cancel_Save_Rate_Engagers - Cancel_Save_Rate_Holdout                                            AS Cancel_Save_Rate_Lift_PTS
       ,safe_divide(Cancel_Save_Rate_Engagers,Cancel_Save_Rate_Holdout) *100                            AS Cancel_Save_Rate_Lift_Index
       ,(Cancel_Save_Rate_Engagers - Cancel_Save_Rate_Holdout) * Total_Cancel_Save_Denom_Targeted     AS Cancel_Save_Rate_Incrementals

       ,Paid_Churn_Rate_Engagers                                                                                    AS EOM_Paid_Churn_Rate_Engagers
       ,Paid_Churn_Rate_Holdout                                                                                     AS EOM_Paid_Churn_Rate_Holdout
       ,Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout                                                          AS EOM_Paid_Churn_Rate_Lift_PTS
       ,safe_divide(Paid_Churn_Rate_Engagers,Paid_Churn_Rate_Holdout) *100                                          AS EOM_Paid_Churn_Rate_Lift_Index
       ,(Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout) * EOM_Paid_Churn_Denom_Targeted                        AS EOM_Paid_Churn_Rate_Incrementals

FROM CTE_2

Query is running:   0%|          |

Downloading:   0%|          |

In [17]:
df_acc = df_acc.sort_values(by='Account_Type')
df_acc

,Report_Month,Account_Type,Email_Engagers,Email_Holdout,Return_Rate_Engagers,Return_Rate_Holdout,Return_Rate_Lift_PTS,Return_Rate_Lift_Index,Returns_Incrementals,Usage_Engagers,...,Cancel_Save_Rate_Engagers,Cancel_Save_Rate_Holdout,Cancel_Save_Rate_Lift_PTS,Cancel_Save_Rate_Lift_Index,Cancel_Save_Rate_Incrementals,EOM_Paid_Churn_Rate_Engagers,EOM_Paid_Churn_Rate_Holdout,EOM_Paid_Churn_Rate_Lift_PTS,EOM_Paid_Churn_Rate_Lift_Index,EOM_Paid_Churn_Rate_Incrementals
2,2023-01-01,Bundled Premium,3214417,122540,0.576203,0.551485,0.024718,104.482109,79454.503648,8.717095,...,0.007890,0.014286,-0.006396,55.226824,-16.214286,1.0,1.0,0.0,100.0,0.0
1,2023-01-01,Free,7842842,359012,0.170364,0.144332,0.026032,118.036195,204165.180456,0.594830,...,0.000604,0.000276,0.000328,218.885922,40.192435,1.0,1.0,0.0,100.0,0.0
0,2023-01-01,Paying SVOD,4894726,172043,0.845716,0.834640,0.011076,101.327015,54213.024186,21.924115,...,0.063803,0.050518,0.013285,126.298063,1275.985751,0.0,0.0,0.0,NaN,0.0


In [13]:
df_total.to_excel('email_kpi_tot_v4_jan23.xlsx', index=False)
df_acc.to_excel('email_kpi_acc_v4_jan23.xlsx', index=False)